# IMDB Sentiment prediction
## 1. Setup


In [18]:
import re
import pandas as pd
from collections import defaultdict
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import string
import stopwords
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from mpi4py import MPI

nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package punkt to /home/mateusz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mateusz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/mateusz/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mateusz/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
df =  pd.read_csv("./IMDB_Dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [3]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df["sentiment"] = df["sentiment"].map({"positive": 1, "negative":0})
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


## 2. Cleanup
### 2.1 HTML

In [6]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    text_without_tags = soup.get_text(separator=' ')
    return text_without_tags

df["review"] = df["review"].apply(lambda x: remove_html_tags(x))
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming te...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


### 2.2 Removing contractions

In [7]:
def remove_contractions(text):
    # This solution is copied from an answer to a stackoverflow question:
    # https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

    # specific
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)

    # general
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

df["review"] = df["review"].apply(lambda x: remove_contractions(x))
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. The filming te...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there is a family where a little boy...,0
4,"Petter Mattei is ""Love in the Time of Money"" i...",1


### 2.3 Punctuation

In [8]:
def remove_punctuation(text):
    text_list_without_punctuation = [letter for letter in text if letter not in string.punctuation]
    text_without_punctuation = ''.join(text_list_without_punctuation)
    return text_without_punctuation

df["review"] = df["review"].apply(lambda x: remove_punctuation(x))
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production The filming tec...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there is a family where a little boy...,0
4,Petter Mattei is Love in the Time of Money is ...,1


### 2.4 Whitespaces

In [9]:
def remove_whitespace(text):
    pattern = re.compile(r'\s+')
    text_without_whitespace = re.sub(pattern, ' ', text)
    return text_without_whitespace

df["review"] = df["review"].apply(lambda x: remove_whitespace(x))
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production The filming tech...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there is a family where a little boy...,0
4,Petter Mattei is Love in the Time of Money is ...,1


### 2.5 Lowercase

In [10]:
def text_to_lower(text):
    return text.lower()

df["review"] = df["review"].apply(lambda x: text_to_lower(x))
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production the filming tech...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically there is a family where a little boy...,0
4,petter mattei is love in the time of money is ...,1


### 2.6 Stopwords

In [11]:
stoplist = stopwords.get_stopwords("english")
stoplist = set(stoplist)

def remove_stopwords(text):
    no_stopwords_list = [word for word in word_tokenize(text) if word not in stoplist]
    no_stopwords_text = " ".join(no_stopwords_list)
    return no_stopwords_text

df["review"] = df["review"].apply(lambda x: remove_stopwords(x))
df.head()

,review,sentiment
0,one reviewers mentioned watching just 1 oz epi...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake thinks zombie...,0
4,petter mattei love time money visually stunnin...,1


### 2.7 Lemmatization

In [17]:
tag_map = defaultdict(lambda : wordnet.NOUN)
tag_map['J'] = wordnet.ADJ
tag_map['V'] = wordnet.VERB
tag_map['R'] = wordnet.ADV

def lemmatize(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = []
    lemmatizer = WordNetLemmatizer()
    for token, tag in pos_tag(tokens):
        lemmatized_token = lemmatizer.lemmatize(token, tag_map[tag[0]])
        lemmatized_tokens.append(lemmatized_token)
    print("Finished")
    return ' '.join(lemmatized_tokens)


def parallel_lemmatize(data_frame=None):
    comm = MPI.COMM_WORLD
    size = comm.Get_size()
    rank = comm.Get_rank()
    print(size, rank)
    # chunk = len(data_frame) //

parallel_lemmatize()

# df.head()

NameError: name 'MPI' is not defined

In [ ]:
sentences = df["review"]
positive_reviews = ' '.join(sentences[df["sentiment"] == 1])
negative_reviews = ' '.join(sentences[df["sentiment"] == 0])
positive_wordcloud = WordCloud(width=800, height=800, background_color="black",
                               min_font_size=12).generate(positive_reviews)

negative_wordcloud = WordCloud(width=800, height=800, background_color="black",
                               min_font_size=12).generate(negative_reviews)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(17, 8))
ax[0].imshow(positive_wordcloud)
ax[0].set_title("Positive Sentiment")
ax[0].axis("off")
ax[1].imshow(negative_wordcloud)
ax[1].set_title("Negative Sentiment")
ax[1].axis("off");

In [ ]:
all_reviews = positive_reviews + negative_reviews

In [ ]:
# TODO maybe add lemmatization?